In [1]:
import PySimpleGUI as sg

In [80]:
class calculadora:
    
    def __init__(self):
        self.operadores_binarios = ['*', '/', '+', '-']
        self.operadores_unarios = ['sqrt', 'e', 'ln', 'cos']
        self.nuevos = False
        self.l = []
        self.reset()
    
    #Definimos los elementos de la ventana
    def reset(self, *args):
        sg.theme('DarkAmber')
        self.name = "Calculadora"   
        self.layout1 = [  [sg.Text('Entrada', font = ("Helvetica", 20)), sg.InputText(font = ("Helvetica", 20), key = "Entrada")]]
        
        self.layout2 = [[sg.Button('+', font = ("Helvetica", 20)), sg.Button('-', font = ("Helvetica", 20))
             , sg.Button('*', font = ("Helvetica", 20)), sg.Button('/', font = ("Helvetica", 20)), sg.Button('cos', font = ('Helvetica', 20))],
             [sg.Button('sqrt', font = ("Helvetica", 20)), sg.Button("e", font = ("Helvetica", 20)),
             sg.Button('ln', font = ("Helvetica", 20))],
             [sg.Button('(', font = ("Helvetica", 20)), sg.Button(')', font = ("Helvetica", 20))],
             [sg.Button('Izquierda', font = ("Helvetica", 20)), sg.Button('Derecha', font = ("Helvetica", 20))]]
        
        self.layout3 = [ [sg.Button('7', font = ("Helvetica", 20)),sg.Button('8', font = ("Helvetica", 20)),
            sg.Button('9', font = ("Helvetica", 20))],
                   [sg.Button('4', font = ("Helvetica", 20)),sg.Button('5', font = ("Helvetica", 20)),
            sg.Button('6', font = ("Helvetica", 20))],
            [sg.Button('1', font = ("Helvetica", 20)),sg.Button('2', font = ("Helvetica", 20)), sg.Button('3', font = ("Helvetica", 20))],
             [sg.Button('0', font = ("Helvetica", 20)),sg.Button('.', font = ("Helvetica", 20))
              ,sg.Button('j', font = ("Helvetica", 20))]]
        
        self.layout4 = [ [sg.Text(size = (30, 4), font = ("Helvetica", 20), key = 'Resultado')]]
        
        self.layout5 = [ [sg.Button('=', font = ("Helvetica", 20), size = (3,5), bind_return_key=True)] ]
        
        self.layout6 = [ [sg.Button('AC', font = ("Helvetica", 20), size = (3,5))] ]
        
        self.layout7 = [ [sg.Button("DEL", font = ("Helvetica", 20), size = (3,5))] ]
        
        self.layout8 = [ [sg.Button('Salir', font = ("Helvetica", 20))] ]
        
        args = list(args)
        #Aqui añadimos los botones nuevos
        if len(args) != 0:
            for i in range(len(args[0])):
                
                self.layout8.append([sg.Button(args[0][i], font = ("Helvetica", 20))])
        
        self.layout = [[sg.Column(self.layout1)], [sg.Text('_'*107)],
        [sg.Column(self.layout2), sg.VerticalSeparator(), sg.Column(self.layout3), sg.VerticalSeparator(),
         sg.Column(self.layout5), sg.VerticalSeparator(), sg.Column(self.layout6), sg.VerticalSeparator(), sg.Column(self.layout7)],
        [sg.Text('_'*107)], [sg.Column(self.layout8)]]
        
        
        
    # Operar es la funcion encargada de interpretar el string de entrada en operaciones numéricas
    def operar(self, x:str):
            
            #Si el argumento es complejo se devuelve de forma normal
            if(self.tipo(x) == complex):
                return self.tipo(x)(x)

            y = x.split() 
            oper = 0
            
            # Vemos cuantas operaciones hay en la expresión, esto para asegurarnos de que no haya un 
            # número complejo que se interprete como operación
            for i in self.operadores_binarios:
                oper += x.count(i)

            # Si el argumento es solo un valor negativo devuelve el valor como numero entero.
            if x.split('-')[0] == "" and oper == 1:
                return self.tipo(x)(x)

            # Contamos los operadores binarios
            l = []
            for i in self.operadores_binarios:
                if y != x.split(i):
                    l.append(i)

            # Si no hay operadores binarios nos saltamos este paso
            if l != []:
                # Buscamos para ver que operadores binarios hay en la expresión
                for i in self.operadores_binarios:
                    temp = x.split(i, 1)

                    # Si temp = y quiere decir que no hay el operador i en la expresión
                    if temp != y:

                        # Limpiamos la entrada
                        temp[0] = self.removeParenthesis(temp[0])
                        temp[1] = self.removeParenthesis(temp[1])

                        # Este if evita que un valor como "-1" se interprete como operación
                        if temp[0] in self.operadores_unarios or temp[1] in self.operadores_unarios:
                            break

                        # Operamos lo argumentos
                        val1 = self.operar(temp[0])
                        val2 = self.operar(temp[1])

                        return (self.f(i, val1, val2))


            # Ahora buscamos los unarios
            for i in self.operadores_unarios:
                temp = x.split(i, 1)

                if temp != y:

                    temp[1] = self.removeParenthesis(temp[1].split(')')[0])

                    val = self.operar(temp[1])

                    return self.f(i, val)

            return self.tipo(y[0])(y[0])
    
    
    # f se encarga de hacer el calculo numérico, ya recibe números y no strings
    def f(self, operacion,*args):

        import numpy as np
        import cmath

        if type(args[0]) == complex:
            args = [complex(i) for i in args]
            #print(args)
        if operacion == '+':
            temp = args[0]
            for i in range(1, len(args)):
                temp += args[i]
        elif operacion == "-":
            temp = args[0]
            for i in range(1, len(args)):
                temp -= args[i]
        elif operacion == 'nada':
            temp = args[0]
        elif operacion == '*':
            temp = args[0] * args[1]
        elif operacion == '/':
            try:
                temp = args[0] / args[1]
            except:
                temp = "Error"
        elif operacion == 'sqrt':
            if args[0] <0 or type(args[0]) == complex:
                temp = cmath.sqrt(args[0])
            else:
                temp = args[0]**(1/2)
        elif operacion == "e":
            temp = np.exp(args[0])
        elif operacion == "ln":
            temp = np.log(args[0])
        elif operacion == "cos":
            temp = np.cos(args[0])
        else:
            temp = self.f_nuevos(operacion, args[0])
        return temp
    
    # f_nuevos se encarga de las operaciones añadidas
    def f_nuevos(self, operacion, valor):
        import numpy as np
        op = 'np.' + str(operacion) + '(' + str(valor) + ')'
        try:
            eval(op)
            return eval(op)
        except:
            sg.Print('LA NUEVA OPERACION NO ESTA EN NUMPY')
            return eval(op)
        
        
    # Función para remover parentesis de un string
    def removeParenthesis(self, x: str) -> str:
        x = x.replace("(", "")
        x = x.replace(")", "")
        return x

    # Esta funcion se encarga de operar primero los parentesis y además de ignorar algunos complejos
    # para no operar entre su parte real e imaginaria
    def separateParenthesis(self, x):
        try:
            if self.tipo(x)(x) == 0:
                return int(0)
        except:
            a = 0
        while(self.hay_operacion(str(x))):
            a = x.find('(')
            b = x.find(")")

            aa = x.find('(', a+1)
            bb = x.find(')', b-1)

            if type(self.tipo(x, True)) == complex:
                return self.tipo(x, True)

            if self.tipo(x[a:bb+1]) == self.tipo(x[bb+2:]) == complex:
                val = self.f(x[bb+1:aa], complex(x[a:bb+1]), complex(x[bb+2:]))
                if val == 0:
                    return 0
                else:
                    return val

            if a == b == -1:
                return self.operar(x)

            else:
                temp = x[a+1:b]
                temp1 = self.operar(temp)

                if type(temp1) == complex:
                    x = str(self.operar(x))
                    if x == 0:
                        return 0
                    else:
                        return x

            x = x.replace('(', '', 1)
            x = x.replace(')', '', 1)
            x = x.replace(temp, str(temp1))
        if self.tipo(x) == complex:
            x += ')'
            x = '(' + x

        if x == 0:
            return 0
        else:
            return x
        
    # hay_operacion identifica si en el argumento aún hay una operación.
    # Por ejemplo en la entrada "-1" no hay operacion aunque si hay un operador en ella
    def hay_operacion(self, x):
        cont = 0
        for i in self.operadores_binarios:
            cont += x.count(i)
        if 2 <= cont:
            return True

        for i in self.operadores_binarios:
            if i in (x):
                if i == '-' and x.split(i)[0] == '':
                    return False

                if (i=='+' or i=='-') and sum(['j' in i for i in x.split('+')]) > 0:
                    return False
                return True

        for i in self.operadores_unarios:
            if i in x:
                return True
        return False
        
        
    # Esta función nos permite ver de que tipo numérico es un argumento de tipo string.
    # Si no es posible convertirlo a numérico devuelve str
    def tipo(self, x, bandera = False) -> type:
        try:
            int(x)
            return int
        except:
            try:
                float(x)
                return float
            except:
                try:
                    complex(x)
                    if bandera:
                        return complex(x)
                    else:
                        return complex
                except:
                    try:
                        X = x.split('+')
                        x = X[1] + '+' + X[0]
                        complex(x)
                        if bandera:
                            return complex(x)
                        else:
                            return complex
                    except:
                        try:
                            X = x.split('-')
                            x = X[1] + '-' + X[0]
                            complex(x)
                            if bandera:
                                return complex(x)
                            else:
                                return complex
                        except:
                            return str
                        
    # Aquí añadimos una operación unaria
    def añadir_unario(self, operador:str):
        self.operadores_unarios.append(operador)
        self.l.append(operador)
        self.nuevos = True
        
        
    # run se encarga de correr todo junto
    def run(self):
        # Cada vez que se corre debemos reiniciar los layouts
        if self.nuevos:
            self.reset(self.l)
        else:
            self.reset()
        temp = 0

        # Create the Window
        window = sg.Window('Calculadora', self.layout)
        sg.theme('DarkAmber')
        # Event Loop to process "events" and get the "values" of the inputs
        operacion = ''
        entrada = ''
        lugar = 'derecha'
        while True:
    
            event, values = window.read(timeout = 1000)
            
            if event == sg.TIMEOUT_KEY:
                entrada = values["Entrada"].replace("__TIMEOUT__",'')
                continue
                
            if event == sg.WIN_CLOSED or event == 'Salir': # if user closes window or clicks cancel
                break
            if event == 'Izquierda':
                lugar = 'izquierda'
                continue
            elif event == 'Derecha':
                lugar = 'derecha'
                continue
            elif event == "DEL":
                entrada = entrada[0:-1]
                window["Entrada"].update(entrada)
                continue
            
            if event != '=' and event != "AC":
                if lugar == 'izquierda':
                    entrada = str(event) + entrada
                else:
                    entrada = entrada + str(event)
                window["Entrada"].update(entrada)
                
            elif event == "AC":
                entrada = ""
                window["Entrada"].update(entrada)
                
                
            elif event == '=':
                try:
                    try:
                        entrada = values['Entrada']
                    except:
                        entrada = ''
                    entrada = str(self.separateParenthesis(entrada))
                    window["Entrada"].update(entrada)
                except:
                    entrada = "Error"
                    window["Entrada"].update(entrada)
                

            if temp == 0:
                temp = 0

        window.close()
        
    def refresh(entrada, values):
        entrada = values[0]
        

In [81]:
calc = calculadora()

In [84]:
calc.run()